<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/quora-pair/quora_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%capture
!pip install kaggle
import numpy as np
import pandas as pd

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
!kaggle competitions download -c quora-question-pairs

 99% 306M/309M [00:14<00:00, 23.4MB/s]
100% 309M/309M [00:15<00:00, 21.5MB/s]


In [9]:
%%capture
!unzip '/content/quora-question-pairs.zip'

In [35]:
pd_train = pd.read_csv('/content/train.csv.zip')

In [37]:
pd_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [41]:
pd_train[100:135]

,id,qid1,qid2,question1,question2,is_duplicate
100,100,201,202,Will there really be any war between India and...,Will there be a nuclear war between India and ...,1
101,101,203,204,Did Ronald Reagan have a mannerism in his speech?,How did Ronald Reagan react to 9/11?,0
102,102,205,206,What were the war strategies of the Union and ...,How could the Confederates have possibly defea...,0
103,103,207,208,Which is the best fiction novel of 2016?,Which are some of the best fiction novels?,0
104,104,209,210,Can I recover my email if I forgot the password?,What should I do if I forgot my email password?,1
105,105,211,212,Will the recent demonetisation results in high...,How demonetisation could affect the GDP of Ind...,0
106,106,213,214,Have you ever heard of travel hacking?,Can WhatsApp be hacked?,0
107,107,215,216,What's the difference between love and pity?,What is the difference between love and pity?,1
108,108,217,218,How competitive is the hiring process at Repub...,How competitive is the hiring process at S & T...,0
109,109,219,220,How Google helps in spam ranking adjustment of...,What is the distribution of traffic between Go...,0


In [57]:
merged_set=[]
for i in pd_train.question1.tolist():
    merged_set.append(str(i))
for i in pd_train.question2.tolist():
    merged_set.append(str(i))
merged_set[:5]

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?']

In [43]:
config = {
    "model_name":'albert-base-v1',
    "batch_size":500,
    "max_seq_length":64,
    "neighborhood_min_size":3,
    "cutoff_threshold":0.95,
    "kmeans_max_iter":100,
    "kmeans_random_state":42,
    "kmeans_no_clusters":8
}

In [44]:
!pip install ClusterTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
import ClusterTransformer.ClusterTransformer as cluster_transformer

In [58]:
def compute_topics(list):
    
    #Instantiate the object
    ct=cluster_transformer.ClusterTransformer()
    #Transformer model for inference
    model_name=config["model_name"]
    
    #Hyperparameters
    #Hyperparameters for model inference
    batch_size=config["batch_size"]
    max_seq_length=config["max_seq_length"]
    convert_to_numpy=False
    normalize_embeddings=False
    
    #Hyperparameters for Agglomerative clustering
    neighborhood_min_size=config["neighborhood_min_size"]
    cutoff_threshold=config["cutoff_threshold"]
    #Hyperparameters for K means clustering
    kmeans_max_iter=config["kmeans_max_iter"]
    kmeans_random_state=config["kmeans_random_state"]
    kmeans_no_clusters=config["kmeans_no_clusters"]
    
    #Sub input data list
    #sub_merged_sent=merged_set[:200]
    sub_merged_sent=list
    #Transformer (Longformer) embeddings
    embeddings=ct.model_inference(sub_merged_sent,batch_size,model_name,max_seq_length,normalize_embeddings,convert_to_numpy)
    #Hierarchical agglomerative detection
    output_dict=ct.neighborhood_detection(sub_merged_sent,embeddings,cutoff_threshold,neighborhood_min_size)
    #Kmeans detection
    output_kmeans_dict=ct.kmeans_detection(sub_merged_sent,embeddings,kmeans_no_clusters,kmeans_max_iter,kmeans_random_state)
    #Agglomerative clustering
    neighborhood_detection_df=ct.convert_to_df(output_dict)
    #KMeans clustering 
    kmeans_df=ct.convert_to_df(output_kmeans_dict)
    return neighborhood_detection_df,kmeans_df 

In [54]:
-len(merged_set)

-808580

In [ ]:
n_df, k_df = compute_topics(merged_set)

Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at albert-base-v1 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.weight', 'predictions.

In [48]:
print(f'DataFrame from neighborhood detection:\n {n_df}')
print(f'DataFrame from Kmeans detection:\n {k_df}')

DataFrame from neighborhood detection:
      Cluster                                               Text
0          0  What would a Trump presidency mean for current...
1          0  What are some tips on making it through the jo...
2          0             What is best way to make money online?
3          0             How should I prepare for CA final law?
4          0      What's one thing you would like to do better?
..       ...                                                ...
114        3  What does it mean that every time I look at th...
115        3                        How do we prepare for UPSC?
116        4        How do I read and find my YouTube comments?
117        4  Why are so many Quora users posting questions ...
118        4  Does the United States government still blackl...

[119 rows x 2 columns]
DataFrame from Kmeans detection:
      Cluster                                               Text
0          6  What is the step by step guide to invest in sh...
1     

In [ ]:
length  = len(n_df)
half = length // 2;
df_question_1 = n_df[0:half]
df_question_2 = n_df[half:]

In [ ]:
question_1 = df_question_1.Cluster.tolist()

question_2 = df_question_2.Cluster.tolist()

duplicate = []
for index range(len(question_1)):
  if question_1[index] == question_2[index]:
    duplicate.append(1)
  else:
    duplicate.append(0)